In [1]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters.filters import * 
from ipyregulus.filters.monitor import Monitor

In [2]:
data = io.load('data/gauss4')
# data = io.load('data/simulations')

In [3]:
from ipywidgets import Output
debug = Output(title='debug')
debug

Output()

### Helper functions

#### view

In [4]:
def update_view(view):
    def _inner(f):
        view.show = filter_tree(view.tree, f)
    return _inner 

In [5]:
def view(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(tree=data, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=update_view(v))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

#### reduce

In [6]:
def reduce_view(widget, tree):
    def _inner(f):
        widget.ref = reduce_tree(tree, filter=f)
    return _inner

In [7]:
def reduce(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data.tree, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=reduce_view(v, data.tree))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

### Initial tree

In [8]:
tw = TreeWidget(data.tree)

In [9]:
v1, f1, p1 = view(tw, title='Initial Tree')

In [10]:
v1.__dict__

{'_trait_values': {'_model_module': '@regulus/ipyregulus',
  '_model_module_version': '1.0.0',
  '_model_name': 'TreeViewModel',
  '_owner': None,
  '_view_count': None,
  '_view_module': '@regulus/ipyregulus',
  '_view_module_version': '1.0.0',
  '_view_name': 'TreeView',
  'field': 'span',
  'title': '',
  'tree': None,
  'comm': <ipykernel.comm.comm.Comm at 0x108d6ecc0>,
  'keys': ['_dom_classes',
   '_model_module',
   '_model_module_version',
   '_model_name',
   '_view_count',
   '_view_module',
   '_view_module_version',
   '_view_name',
   'attrs',
   'details',
   'field',
   'layout',
   'selected',
   'show',
   'title',
   'tree_model'],
  '_dom_classes': (),
  'attrs': {},
  'details': [],
  'layout': Layout(),
  'selected': [],
  'show': set(),
  'tree_model': None,
  '_display_callbacks': <ipywidgets.widgets.widget.CallbackDispatcher at 0x1157f69b0>},
 '_trait_notifiers': {'comm': {'change': [<traitlets.traitlets.ObserveHandler at 0x108deda20>]}},
 '_trait_validators': {

In [11]:
v1.super()

AttributeError: 'TreeView' object has no attribute 'super'

In [21]:
v2, f2, p2 = view(tw, attr='fitness', title='fitness')

In [22]:
f2.func = lambda x, v: v > x

In [23]:
v2.attr = 'parent_fitness'

In [24]:
f2.attr = 'span'
f2.func = lambda x, v: v <=x

### Reduce tree

In [29]:
v3, r3, p3 = reduce(data, attr='span', title='reduce')

In [30]:
v3.attr = 'parent_fitness'

In [32]:
v3.tree

In [33]:
data.tree

In [34]:
v3.ref

### fitness

#### test

In [35]:
tw2 = TreeWidget(data.tree)

In [36]:
f4 = AttrFilter(attr='span', func=lambda x, v: v <= x)

In [37]:
def test(f):
    tw2.ref = reduce_tree(tw.tree, filter=f)

In [38]:
m4 = Monitor(f4, func=test)

In [39]:
v4 = TreeView(tw2, attr='span') 

In [40]:
p4 = SidePanel(title='Test')
with p4:
    display(v4, f4)

In [41]:
fitness = view(tw2, attr='fitness', title='fitness')

In [42]:
parent_fitness = view(tw2, attr='parent_fitness', title='parent_fitness')

### Data widget

In [43]:
dw = DataWidget(data=data)

In [44]:
details = DetailsView(data=dw)

In [45]:
p7 = SidePanel(title='details')
with p7:
    display(details)

In [ ]:
widgets.dlink((fitness[0], 'details'), (details, 'show'))

In [ ]:
@debug.capture()
def test(nv, av):
    print('test', nv, av)
    return nv < av

In [ ]:
a = AttrFilter(attr='span', func=test)

In [ ]:
display(a)

In [ ]:
a(tw.tree, tw.tree.root)

In [ ]:
@debug.capture()
def test2(*args, **kwargs):
    print('test2', *args, **kwargs)

In [ ]:
m = Monitor(a, func=test2)

In [ ]:
a.observe(test2, names='changed')

In [ ]:
a.changed

In [ ]:
a.ui

In [ ]:
a.ui.__dict__

In [ ]:
a.ui.observe(test2, names='value')

In [ ]:
if None in (8, None):
    print('yes')